In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '../')


In [ ]:
from tqdm import tqdm
import json
from pathlib import Path
from mlflood.conf import PATH_GENERATED
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mlflood.preprocess import build_dataset


In [ ]:
catchment_num = "toy"
# catchment_num = "709"

# catchment_num = "709_new"
# catchment_num = "684"

# catchment_num = "709wall"
# catchment_num = "709wall_0threshold"
# catchment_num = "709_0threshold"

# Creating the h5 files

In [ ]:
build_dataset(catchment_num)

# Analysing the dataset

In [ ]:
name = "train"
with h5py.File(PATH_GENERATED / Path(catchment_num + "-" + name+".h5"), 'r') as f:
    dem = f["dem"][()]
    mask = f["mask"][()]
    event_name = f["event_name"][()]
    timesteps = []
    waterdepth = []
    rainfall_events = []
    for k in filter(lambda x: "timesteps"==x[:9],  sorted(f.keys())):
        timesteps.append(f[k][()])
    for k in filter(lambda x: "waterdepth"==x[:10],  sorted(f.keys())):
        waterdepth.append(f[k][()])
    for k in filter(lambda x: "rainfall_events"==x[:15],  sorted(f.keys())):
        rainfall_events.append(f[k][()])


In [ ]:
# waterdepth = np.concatenate(waterdepth)
# rainfall_events = np.concatenate(rainfall_events)
# timesteps = np.concatenate(timesteps)

In [ ]:
plt.imshow(dem)
plt.colorbar()

In [ ]:
plt.imshow(mask)
plt.colorbar()

In [ ]:
k = 24
for t, re in zip(timesteps, rainfall_events):
    plt.plot(t[:k], re[:k])
plt.title("Rainfall events")
plt.xlabel('Timesteps')
plt.ylabel('Rainfall (mm)')

In [ ]:
def print_stat(x):
    print(np.min(x), np.max(x), np.mean(x), np.std(x))

In [ ]:
waterdepth = [w[:,mask.astype(bool)] for w in waterdepth]

In [ ]:
print_stat(np.concatenate(waterdepth))

In [ ]:
plt.hist(np.concatenate(waterdepth).flatten(), 100);
plt.yscale("log")

In [ ]:
diff_waterdepth = [np.diff(w, axis=0) for w in waterdepth]

In [ ]:
print_stat(np.concatenate(diff_waterdepth))

In [ ]:
plt.hist(np.concatenate(diff_waterdepth).flatten(), 100);
plt.yscale("log")

In [ ]:
[(np.mean(w[-1,:]) - np.mean(w[0,:]))/(w.shape[0] -1) for w in waterdepth]